# Overview

For each assignment data stored in \Graph\find_cycles\examples\graphname\graphname.pkl

we produce \Graph\find_cycles\Done_examples\graphname\graphname.txt and .pkl

which stores the data of the cycle on each subgraph T \cup e of each weak stability condition b(y undoing the process we used to get all assignments).

# Table of contents

1. [Functions](#s1)
    1. [Load data: Initialisation: get assignment data for a given subgraph](#s11)
    2. [Remove tails of trees in subg](#s12)
    3. [Relabel trees to In format using common_tree](#s13)
    4. [Translate all assignments in subg_datum by common_tree_ass so that for common_tree_ass=0](#s14)
    5. [Remove components from vertices of tail of subg from assignments of trees in subg](#s15)
    6. [get_Im_data: Put all above functions together now](#s16)
    7. [Check against g=1 case](#s17)
    8. [Put cycles into correct format wrt G edge labeling](#s18)
    9. [Running over all cases for a given graph](#s19)
    10. [Saving data](#s10)
2. [Determine the cycles for the following](#s2)

# Functions <a name="s1"></a>

## Load data: Initialisation: get assignment data for a given subgraph<a name="s11"></a>

In [7]:
def get_subgraphs_gm1_Edgesmissing(G):
    
    #Get all spanning subgraphs, on row above trees specifically, $\tilde{\Gamma}_{g-1}$.
    #returns a list of such subg
    
    edge_num_G=len(G.edges())
    vert_num_G=len(G.vertices())
    g=edge_num_G-vert_num_G+1
    gm1=g-1

    E=Set(G.edges()) #remeber possibly mulitgraph
    V=G.vertices()

    graphs_i_want=[]
    for s in E.subsets():
        if len(s)==edge_num_G-gm1: # edge_num_G-gm1number of edges of these subgraphs.
            H=Graph(multiedges=True)
            H.add_edges( s )
            H.add_vertices( V )        # are interested only in spanning subgraphs
            if H.is_connected(): #Want connected subgraphs
                graphs_i_want.append(H)
    return graphs_i_want

In [8]:
def unpickle_assignment_datum(graphname):
    # Unpickle the list
    filename=f"examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

In [9]:
def get_tree_ass_data(ass_datum,ordering_trees):
    #Take only the assignment data for all
    ass_data_only=[data[0] for data in ass_datum]

    #Attach to each assignment the appropriate tree for all wsc.
    assignment_data_all=[]
    for wsc_ass_data in ass_data_only:
        #wsc_ass_data is a list of assignments for each tree, and which determines a weak stab con after chip-adding
        temp_data=[]
        for index,ass in enumerate(wsc_ass_data):
            temp_data.append((ordering_trees[index],ass))
        assignment_data_all.append(temp_data) 
    return assignment_data_all

In [10]:
l = [get_subgraphs_gm1_Edgesmissing, unpickle_assignment_datum, get_tree_ass_data]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Remove tails of trees in subg<a name="s12"></a>

In [11]:
def get_tails(subg):
    #tails = edges with vertices outside of subnecklace. 
    #return multigraph of tails after removing edges of subnecklace.

    subg_edges=subg.edges()
    #Subnecklace edges
    [edges_subnecklace]=subg.cycle_basis(output='edge')

    #Get tails
    tails_subg=copy(subg)
    tails_subg.delete_edges(edges_subnecklace) #order of the edges must matter.

    return tails_subg

In [12]:
def remove_discon_verts(G):
    isolated = [v for v in G.vertices() if G.degree(v) == 0]
    G.delete_vertices(isolated)
    return G

In [13]:
def remove_tails_tree_of_subg(data,tails):#For get_ass_trees_subg
    """
    Obj:For the trees of subg we remove the tails of the subgraph relative to the subnecklace (dont bother assingments yet).
    Return: list of (T,assT) where T does not have tails of subg.
    """
    temp_data =[] # temp_data for subnecklace.
    for d in data: #data of In assignments (tree,ass) pairs
        tree_subg=copy(d[0])
        tree_subg.delete_edges(tails.edges())
        tree_subg=remove_discon_verts(tree_subg)
        temp_data.append((tree_subg,d[1]))
    return temp_data

In [14]:
l = [get_tails, remove_discon_verts, remove_tails_tree_of_subg]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Relabel trees to In format using common_tree <a name="s13"></a>

In [15]:
def find_leaves(spanning_tree):#For get_ass_trees_subg
    """
    Obj: to get relabeler, need to define a start and end for the path 
    in the subnecklace of subg wrt common_tree (specifically fixed) after we have removed tails.
    #returns the leaves of tree in subnecklace.i.e [start,end]
    
    We should have alreadt removed the edges of tails from subg tree to get spanning_tree.
    
    """
    nodes=spanning_tree.vertices()
    leaves = []
    for node in nodes:
        edges = [x for x in spanning_tree.edges() if node in x]
        if len(edges) == 1:
            leaves.append(node)
#     show(spanning_tree) #---#
#     print(leaves) #---#

#     return sorted(leaves)
    return leaves

In [16]:
def get_relabler(subg,Im_common_tree,leave_order=None):
    
    """
    Obj: Get relabling, that is given the path in subnecklace we need a method to 
    relabel the vertices of trees of I_n to the appropriate ones in the subnecklace of sugb.to get relabling we use
    common tree in subg (acting as 0 assignment).
    #returns a dictionary mapping the vertices of the subnecklace to In. 
    
    The specific choice of leave_order does not matter.
    
    Returns a mapping of vertices of In to vertices of subnecklace.
    i.e a dictionary for mapping.
    """
#     print("subg")#---#
#     show(subg)#---#
    
    
#     print("Im_common_tree")#---#
#     show(Im_common_tree)#---#
    
    [start,end]=find_leaves(Im_common_tree)
    [path]=Im_common_tree.all_paths(start, end) #of the form [[0, 5, 1, 4, 2, 3]]

    relabler={}    
    for i in range(len(path)):
        """
        The End of the subnecklace should map to v_0 of In
        The Start of the subnecklace should be v_1.
        """
        # relabler[i]=path[i-1]
        
        relabler[path[i-1]]=i # 1:0,3:1,2:3,5:4 i think
    return relabler

In [17]:
def get_temp_data(relabler,data):#For get_ass_trees_subg
    """
    Obj:Apply relabler: #relabel tree of subg to trees of Im format v0,..,vn-1.
    Returns:data but with trees relabled.
    """
    temp_data =[] # temp_data for subnecklace.
    for d in data: #data of In assignments (tree,ass) pairs
        tree_In=copy(d[0])
        tree_In.relabel(relabler)
        temp_data.append((tree_In,d[1]))
    return temp_data

In [18]:
l = [find_leaves, get_relabler, get_temp_data]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Translate all assignments in subg_datum by common_tree_ass so that for common_tree_ass=0.<a name="s14"></a>

In [19]:
def un_translate_data(temp_data,common_tree_assingment):
    """we un-extend assignments to subg tails."""
    # Returns: a [(T,Ass_T) | .. ] where T is a tree of G where Ass_T=0 for common_tree_ass
    trans_data= [(tree,np.array(arr) - np.array(common_tree_assingment)) for tree, arr in temp_data]
    return trans_data

## Remove components from vertices of tail of subg from assignments of trees in subg<a name="s15"></a>

In [20]:
def get_Im_assignments(relabler,data,Im_common_tree):

    """Obj:We are reducing the temp assignment on subg tree to assignments on Im.
     """

    vert_num_Im=len(Im_common_tree.vertices())
    temp_data=[]
    for d in data:
        assignment=d[1] # untranslated assignments of sub on T.
#         print(assignment)
        temp_ass=Im_assignment(assignment, relabler,vert_num_Im) #Assignment of Im  after removing tails vertices and relabeling
#         print(temp_ass)
#         print("\n")
        temp_data.append((d[0],temp_ass))
    
    return temp_data

In [21]:
def Im_assignment(l, relabeling,vert_num_Im):#for get_Im_assignments

    """
    Obj:Get Assignments of Im after removing tails vertices and relabeling
        By changing these assignments to ones on the correct vertices of Im using relabel_dict.
        Then excluding the vertices of the tails of subg.
    Inputs:    
        # l #assignment on tree from subg #l = and assignemnt of length n.
        # relabeling # relabel dict from subgraph to Im
    Returns: 
        # Non-tranlated assignment on T in Im.
    """
#     print(relabeling)
    result = [0] * vert_num_Im # number of vertices of $subnecklace$.
    for i, value in enumerate(l):
#         print(i, value)
        # is i a key of the relabling?
        if str(i) in relabeling: # if so then do
#             print("test")
#             print(int(relabeling[str(i)]))
            result[int(relabeling[str(i)])] = value
        
                # relabler[path[i-1]]=i # 1:0,3:1,2:3,5:4 i think
#             print(result)
    return np.array(result)

In [22]:
l = [get_Im_assignments, Im_assignment]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## get_Im_data: Put all above functions together now<a name="s16"></a>

In [23]:
def standardised_edge_order(edges_Im,vert_Im): #For get_Im_data
    
    """ For:
        def get_Im_data(case,subg): #updated
            The following function returns the assignment data we want on Im part for the subnecklace.
            Using this we can get the cycle.

            Returns: relabler too 
        # Pick subg
        vert_num_G=len(list(subg.vertices()))

        ## Remove disconnected vertices from tails.    
        tails=get_tails(subg)
        tails=remove_discon_verts(tails)

        # What is Im necklace
        [edges_subnecklace]=subg.cycle_basis(output='edge') #return cycle of subgnecklace
        print("edges_subnecklace",edges_subnecklace)
        m=len(edges_subnecklace) #length of subnecklace

        #We need to order the edges of edges_subnecklace: so that spanning trees of Im are in the correct order
        temp_Im=Graph(edges_subnecklace)
        verts_Im=set(temp_Im.vertices())
        #     print(verts_Im) #["0","2",..]
        edges_subnecklace=standardised_edge_order(edges_Im,vert_Im)

        Im=Graph(edges_subnecklace)
    """
    
    """
        edges_Im=[('4', '3', None), ('3', '2', None), ('2', '0', None), ('0', '4', None)]
        vert_Im = {"0", "2", "3", "4"}
        standardised_edge_order(edges_Im,vert_Im)
        # [('0', '2', None), ('0', '4', None)]
        # [('2', '3', None)]
        # [('3', '4', None)]
        #     print(edge_lst) #[('0', '2', None), ('0', '4', None), ('2', '3', None), ('3', '4', None)]

    """
    

    # Sort the edges in sorted_lst
    sorted_lst = []
    for tpl in edges_Im:
        sorted_tpl = tuple(sorted(tpl[:-1])) + (tpl[-1],)
        sorted_lst.append(sorted_tpl)

    # Filter the tuples to only include those where the first element is in vert_Im
    filtered_lst = [tpl for tpl in sorted_lst if tpl[0] in vert_Im]

    # Group the tuples by their first element
    grouped_tuples = {}
    for tpl in filtered_lst:
        if tpl[0] not in grouped_tuples:
            grouped_tuples[tpl[0]] = []
        grouped_tuples[tpl[0]].append(tpl)

    # Sort each list of tuples by their second element
    for key in grouped_tuples:
        grouped_tuples[key] = sorted(grouped_tuples[key], key=lambda x: x[1])

    # Extract the sorted lists of tuples as a list and sort the list
    sorted_lists = sorted(list(grouped_tuples.values()))

    # Print the result
    edge_lst=[]
    for lst in sorted_lists:
#         print(lst)
        edge_lst=edge_lst+lst
        
    return edge_lst

In [24]:
def get_Im_data(case,subg): #updated
    """
        The following function returns the assignment data we want on Im part for the subnecklace.
        Using this we can get the cycle.

        Returns: relabler too 
    """
    # Pick subg
    vert_num_G=len(list(subg.vertices()))
    
    ## Remove disconnected vertices from tails.    
    tails=get_tails(subg)
    tails=remove_discon_verts(tails)
    
    # What is Im necklace
    [edges_subnecklace]=subg.cycle_basis(output='edge') #return cycle of subgnecklace
#     print("edges_subnecklace",edges_subnecklace)
    m=len(edges_subnecklace) #length of subnecklace

    #We need to order the edges of edges_subnecklace: so that spanning trees of Im are in the correct order
    temp_Im=Graph(edges_subnecklace)
    vert_Im=set(temp_Im.vertices())
    #     print(verts_Im) #["0","2",..]
    edges_subnecklace=standardised_edge_order(edges_subnecklace,vert_Im)
    Im=Graph(edges_subnecklace) #KEY!
    

    # Get spanning trees of subg
    subg_trees=list(subg.spanning_trees())

    #Data we want to analyse.# Pick (T,Ass) from ass_datum for trees in subg_trees.
    data=[]
    for pair in case:
        if pair[0] in subg_trees:
            data.append(pair)
    
    """
        Note: common_tree,common_tree_assingment are the tree assignment pair we translate by.

        To avoid making a choice of spanning tree for each subg i.e.
        common_tree,common_tree_assingment=data[0] # tree with tails still

        We make a choice of spanning tree of Im. This is important as subg with the same Im shoudld have the same cycle.

        At the moment we believe that
        Im_tree=list(Im.spanning_trees())[0]
        will always choose the same tree for each tree containing the same Im.

        Then for each subg we add back the tails to get common_tree.

        Then need  to:
        ## Pick (t,ass_t) where we will have ass_t=0 for common_tree.
        common_tree_assingment=data[0] # tree with tails still

    """
    #choice of spanning tree of Im necklace. (do we need to relable? dont think so)
    Im_common_tree=list(Im.spanning_trees())[0] #KEY!
    
    #WHY SPANNING TREES OF IM NOT ORDERED THE SAME ???????
    
    #print the trees 
#     sp_im=Im.spanning_trees()
#     print("Im.edges()",Im.edges())
#     print("subg Case:",[t.edges() for t in sp_im],"\n")
    
    
    #Add back tails to Im_tree.
    common_tree=Graph(multiedges=True)
    common_tree.add_edges(Im_common_tree.edges())
    common_tree.add_edges(tails.edges()) # tree with tails still

    ## Pick (t,ass_t) where we will have ass_t=0.    
    for pair in data:
        if common_tree.edges()==pair[0].edges(): #CHECK i think its enough to just give the edges here
            common_tree_assingment=pair[1]

    #Now we have common_tree,common_tree_assingment

    # From data we need to remove the tails from the trees. Remove tails from trees of subg in data.
    removed_tails_data=remove_tails_tree_of_subg(data,tails)
    data=removed_tails_data
    
    """  We now need to get Im_common_tree_assingment. Label (t,ass_t) where we have ass_t=0 in Im.
        This will not be: Im_common_tree,Im_common_tree_assingment=data[0]
        #but as we have Im_common_tree already and the not changing the assignment here we just have,"""
    Im_common_tree_assingment=common_tree_assingment

    ## Relabel trees of data in Subg necklace format to In format.
    relabler=get_relabler(subg,Im_common_tree,leave_order=None)
    temp_data=get_temp_data(relabler,data)


    # Consider the assignments and translate so common_tree_assingment=0.
    Im_data=un_translate_data(temp_data,common_tree_assingment)

    ## Remove components of assignments from vertices of tail of subg from assignments of trees in subg.

    Im_data=get_Im_assignments(relabler,Im_data,Im_common_tree)

    # We record the common_tree to store with data.
    translator_tree=common_tree
    
    return Im_data,relabler,translator_tree

In [25]:
l = [standardised_edge_order, get_Im_data]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Check against g=1 case<a name="s17"></a>

In [26]:
## Weak_stability_genus_1

#For change_to_get_trees, produces stability conditions for genus 1 graph given by a cycle.
def head(n, prev_tree_edge):
    """
    n= number of vertices in I_n
    previous element of the cycle, ie tree_edge is j where e_j ={v_j,v_{j+1}} 
    
    #Also used in chip adding on necklace curve
    """
    head = np.zeros(n)

    if prev_tree_edge == n:
        head[0] = 1
    else:
        head[prev_tree_edge] = 1

    return head
def tail(n, current_tree_edge):
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = -1

    return tail
def tail_plus(n, current_tree_edge):  #for chip adding
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = 1

    return tail

def cycle_to_assignments(cycle):
    #Main to get assingments from cycle

    n = len(cycle)

    # cycle=(1,3,4,2)
    ass_0 = np.zeros(n)

    M = [(1, ass_0)]  # Memory

    for i in range(1, len(cycle)):

        current_tree_edge = cycle[i]
        tup = M[-1]  #pick last item of M

        vtail = tail(n, current_tree_edge)
        vhead = head(n, tup[0])
        prev_ass = tup[1]

        current_ass_t = prev_ass + vtail + vhead
        M.append((current_tree_edge, current_ass_t))

    # for i in M:
    #     print(i)

    return M
def change_to_get_trees(cycle):
    
    #return trees where there was previously just numbers indicating edge to be removed.
        # change: [(1, array([0., 0., 0., 0.])), (3, array([ 0.,  1., -1.,  0.])), (2, array([ 0.,  0., -1.,  1.])), (4, array([0., 0., 0., 0.]))]
    # so first term is tree of In
    # cycle=(1,3,2,4)

    data_In=cycle_to_assignments(cycle) #<-----------------
    data_In=sorted(data_In, key=lambda x: x[0]) # reordering to (1,),(2,) ect.. instead of being ordered by cycle.
    
    """
    Wish to order data_In by first term of tuple so that is (1, ),(2,) thus putting trees in order so that when we append to Memory M we have
    M'[counter](0) = M[counter][0] when we are iterating through counters.
    """
    
    n=len(cycle)
    G=graphs.CycleGraph(n)
    
    new_data_In=[]
    for data in data_In:
        #Get tree
        tree_info=data[0]
    #     print(tree_info)
        if tree_info== len(data_In):
            edge_tree=(tree_info-1,0)
        else:
            edge_tree=(tree_info-1,tree_info)
    #     print(edge_tree)
        tree=copy(G)
        tree.delete_edge(edge_tree)

        #Recompile data
        new_data_In.append((tree,data[1]))

    # plot(new_data_In[2][0]) #Have the correct trees.
    
    """
    Example:x=change_to_get_trees(cycle)

    print(cycle)
    for i in x:
        print(i[0].edges())
        
        cycle=(1,3,2)
    x=change_to_get_trees(cycle)
    print([tup[0].edges() for tup in x])
    """
    
    return new_data_In

def get_cycles(n):
    lst = list(range(1, n + 1))
    x = permutations(lst, n)
    y = [i for i in x if i[0] == 1]

    #Does:
    # x=permutations([1,2,3,4,5],5)
    # y=[i for i in x if i[0]==1]
    # # print(len(y))
    # for i in y:
    #     print(i)

    return y

In [27]:
def get_ordered_list(M, ordering_trees):# for get_assignment_ordered_for_wstab
    """
    We now have the subnecklace in the format of In. 
    We dont have a method to go from assignments to necklaces. 
    What we can do is run through all cycles on In and ask which cycle produces the assignment data we have,
    it is then this cycle that we take.
    
    #Given M, a list of (t,ass) produce a list of (t',ass') whose ordering is dependant on a specified ordering of trees ordering_trees.
    #M: list of (T,Ass_T).
    #returns list of (t',ass') ordered wrt ordering_trees. 
    """
    P = []
    mapping = {t.copy(immutable=True): ass for (t, ass) in M}
    for t in ordering_trees:
        P.append((t.copy(immutable=True),mapping[t.copy(immutable=True)]))
    return P

In [95]:
def check(Im_data,n):
    """
    Obj: given Im_data checks which cycle gives this data in the In case and returns the cycle.
    return:Given list of $(T^{'},Ass(T^{'}))$ for subg wish to get cycle.
    """
    cycles=get_cycles(n)
    
    good_cycles=[]
    for cycle in cycles:
        cycle_data=change_to_get_trees(cycle)
        cycle_trees=[d[0] for d in cycle_data]
        cycle_assignments=[d[1] for d in cycle_data]

        #Put Im_data into In case format.
        Im_data=get_ordered_list(Im_data,cycle_trees)
        Im_data_assignments=[d[1] for d in Im_data]
        
        equal = all(np.array_equal(x, y) for x, y in zip(cycle_assignments, Im_data_assignments))
        if equal:
            good_cycles.append(cycle)
        else:
            pass
    
    if len(good_cycles)==1:
        [cycle]=good_cycles
        return cycle
    else:
        ##!!
        print("Something is wrong")
        print(f"len(good_cycles):{len(good_cycles)}")
        
        #Let take the first one if more than one.
        
        cycle=good_cycles[0]
        return cycle

In [29]:
l = [head, tail,tail_plus,get_cycles,change_to_get_trees,cycle_to_assignments,get_ordered_list]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Put cycles into correct format wrt G edge labeling<a name="s18"></a>

In [60]:
#Correcting cycles of Im to cycles for subg with correct labeling.

def get_edge_subg_for_i_in_cycle(i,cycle,swapped_relabler):#for get_relabled_cycle
    """Given element of cycle, how do we return the edge of In, format is standardised.
    for i , take ("i-1","i")"""
    
    print(f"i:{i}:{type(i)}")##!!
    
    edge=(str(i-1),str(i)) # edge of Im 
    
    # Need to use relable to convert to edge of subg.
    subg_edge=[]
    for e in edge:
        if int(e)==len(cycle):
            subg_edge.append(swapped_relabler["0"])
        else:    
            subg_edge.append(swapped_relabler[e])
    subg_edge=sorted(subg_edge)
    return subg_edge
def get_labeled_edge_subg(subg_edge,G_edges_labeled):#for get_relabled_cycle
    """
    Given a edge of subg of G take the label of G_edges_labeled.
    Returns "e4" of G
    """
    for t in G_edges_labeled:
        if t[0] == subg_edge[0] and t[1] == subg_edge[1]:
            extracted_value = t[2]
            break
    return extracted_value
#sub-Main for here
def get_relabled_cycle(cycle,G_edges_labeled,swapped_relabler):

    """
    Obj:Given a cycle for Im, relables the components wrt the lableing of G.
    
    Inputs:
    -cycle: cycle of Im.
    
    Returns:
    """
    relabled_cycle=[]
    for i in cycle:
        subg_edge=get_edge_subg_for_i_in_cycle(i,cycle,swapped_relabler)

        subg_edge=tuple(subg_edge)

        #Once we have the edge of subg that correcponds to the element (edge of Im) 
        #of the cycle we ned to take the lableing of the edge in G.

        new_cycle_element=get_labeled_edge_subg(subg_edge,G_edges_labeled)

        new_cycle_element=int(new_cycle_element[1]) # for "e5" take 5.

        relabled_cycle.append(new_cycle_element)

    relabled_cycle=tuple(relabled_cycle)
    return relabled_cycle

def apply_edge_labels(G,edge_labels):# for get_labeled_subg_cycle_data
    """How to label edges of G"""
    # Define the graph G
    # Define the edge labels
    # # Set the edge labels
    for i in edge_labels.items():
        tup=i[0]+(i[1],)
        G.set_edge_label(*tup)
    return G

def add_edges_to(graph,G_edges_labeled):# for get_labeled_subg_cycle_data
    """Adds labelings of edges of G to subgraph :should also work for trees."""
    subg=graph
    # Add edge labels to subg
    subg_edges_labled=[]
    for subg_edge in subg.edges():
        #Get the lable and construct new edge with label
        e_label=get_labeled_edge_subg(subg_edge,G_edges_labeled)
        subg_edge=(subg_edge[0],subg_edge[1],e_label)
        subg_edges_labled.append(subg_edge)
    subg=Graph(subg_edges_labled, multiedges=True)
    return subg

#Main
def get_labeled_subg_cycle_data(G,edge_labels,subg_data):
    """Obj #Relabel the cycles and subg data to labeling of G:"""
    
    G=apply_edge_labels(G,edge_labels)
    G_edges_labeled=G.edges()

    storage=[] # like of (subg,cycles) after edge labeling and modifiying cycles to this labeling.
    for i in subg_data:

        #unpacket data
        subg_cycle_pair,relabler,translator_tree=i #(subg,cycle) pair and relalber for subg to In format.
        subg,cycle=subg_cycle_pair

        #To go from Im to necklace of subg vertex labeling
        swapped_relabler = {str(value):key for key, value in relabler.items()}

        new_cycle=get_relabled_cycle(cycle,G_edges_labeled,swapped_relabler)
        
#         print(f"old cycle {cycle}, new_cycle: {new_cycle}") #---#
#         print(f"cycle: {new_cycle} \n Subg: {subg.edges()} \n") #---#
#         print(f"cycle: {new_cycle}") #---#


        #add lalbeings to edges of subg
        subg=add_edges_to(subg,G_edges_labeled)

        #store new data
        storage.append((subg,new_cycle,translator_tree))
    return storage

In [31]:
# l = [get_edge_subg_for_i_in_cycle,add_edges_to,apply_edge_labels,get_relabled_cycle,get_labeled_edge_subg,get_labeled_subg_cycle_data]
# find_functions_used_l(l)

## Running over all cases for a given graph<a name="s19"></a>

In [62]:
def run_over_all_subgraphs(case,subgraphs):
    """
    Returns: A cycle for each subgraph for a given case "assignment data on trees".
    Record as a list of pairs (subg,cycle) and save using pkl.
    """
    
    subgraphs_with_cycle=[]
    for num in range(len(subgraphs)):
        # Pick subg
        subg=subgraphs[num]
        
        #What is Im necklace
        [edges_subnecklace]=subg.cycle_basis(output='edge') #return cycle of subgnecklace
        Im=Graph(edges_subnecklace)
        m=len(edges_subnecklace)

        Im_data,relabler,translator_tree=get_Im_data(case,subg)
        cycle=check(Im_data,m)
        
        print(f"cycle: {cycle}")
#         print(cycle)
        
        subgraphs_with_cycle.append(((subg,cycle),relabler,translator_tree))
#     print("--- different wsc")
    return subgraphs_with_cycle

In [33]:
def get_subgraph_cycles_for_ass_datum(ass_datum,G,edge_labels,ordering_trees,subgraphs):
    """
    Obj: for each element of ass_datum i.e a list of assignments for a stability condition.
    We return a list of (subgraph,cycle) used to generate assignments for trees contained.
    """
    assignment_data_all=get_tree_ass_data(ass_datum,ordering_trees)
    
    subg_cycle_data=[]
    for j in range(len(assignment_data_all)):
        case=assignment_data_all[j]
        
        #Get (subg,cycle),relabler) without labeling of G
        subg_data=run_over_all_subgraphs(case,subgraphs)
        
        #returns (subg,cycles) with labeling
        subg_cycle_pair_l=get_labeled_subg_cycle_data(G,edge_labels,subg_data) #New subg,cycle pair list
        subg_cycle_data.append(subg_cycle_pair_l)
#         print("--- different wsc") #---#
        
    return subg_cycle_data

In [34]:
#MAIN #use get_subgraph_cycles_for_ass_datum
def run_over_all_assignments(ass_datum,G,edge_labels,ordering_trees,subgraphs):
    
    #We save all of the data together in the order:
    #1. list of only assignments (ordered by spanning trees) # in the correct format for getting lbm
    #2. lbm data
    #3. list of (tree,ass_t) # in the correct format for getting (subg,cycle) data.
    #4. list of (subg,cycle)
    
    assignment_data_all=get_tree_ass_data(ass_datum,ordering_trees)
    subg_cycle_data=get_subgraph_cycles_for_ass_datum(ass_datum,G,edge_labels,ordering_trees,subgraphs)

    total_data=[]
    
    for index,ass in enumerate(ass_datum):

        assignment_l=ass_datum[index][0] #list of assignments as arrays (no trees)
        lbm=ass_datum[index][1] # array of line bundle multidegree data
        case=assignment_data_all[index] #list of (tree,assignments)
        l_subgraph_cycle=subg_cycle_data[index] # list of (subg,cycles) after relabeling
        
        total_data.append((assignment_l,lbm,case,l_subgraph_cycle)) #data described as above.
        
    return total_data

In [35]:
l = [run_over_all_subgraphs, get_subgraph_cycles_for_ass_datum, run_over_all_assignments]
find_functions_used_l(l)

NameError: name 'find_functions_used_l' is not defined

## Saving data<a name="s110"></a>

In [36]:
def pickle_data(graphname,data):
    # Pickle the list
    filename=f"{graphname}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    return

In [37]:
def unpickle(graphname):
    # Unpickle the list
    filename=f"Done_examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

In [38]:
#Storing files 
def output_to_file(G,total_data,ordering_trees):#in text files
    """
    Return to text file:
    - add list of subgraphs under trees.
    - record list of cycles for subg, after lbm. 
    """

    print(f"Information: Here are all stability conditions for the graph G")
    print(f"Graph: {G.edges()} \n")
    
    print(f"There are {len(total_data)} stability conditions for G. \n")
    
    print(f"The spanning trees of G are ordered as follows:")
    for index,tree in enumerate(ordering_trees):
        print(f"Tree {index+1}: {tree.edges()}" )
        
    print(f"\n The spanning subgraphs of G of the for T cup e are ordered as follows:")
    
    """
    Fix order of subg printed:
    
    Here get_subgraphs_gm1_Edgesmissing does not reflect the ordering of subg in subg_cycle_l=tup[3]
    subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs Tcup e.-D

    #Assuming subg_cycle_l for each element of total_data is ordered the same subg.
    """

    tup_example=total_data[0]
    subg_cycle_l=tup_example[3]
    subg_ll=[pair[0] for pair in subg_cycle_l]

    for index,subg in enumerate(subg_ll):
        print(f"Subgraph {index+1}: {subg.edges()}" )
    

    print("\n We will now list all stabilty conditions. The data will be presented in three lines:")
    print("   1. The assignment datum on trees the order of which will be given by the Tree order,")
    print("   2. The union of sets of the form assignment datum of T + the set of break divisors for T, " )          
    print("   3. The cycle used with th subgraph to obtain the assignments on trees (ordered by subgraphs above).\n")

    for tup in total_data:
        
        print(tup[0]) #assignment datum
        
        #Put sigma to list format
        print(tup[1].tolist()) #lbm
        
        subg_cycle_l=tup[3]
        l_cycles=[(x[1],x[2].edges()) for x in subg_cycle_l] #records cycle and tree for sub.
        print(l_cycles)
        
        print("\n")
    
    return

def store_data(G,total_data,ordering_trees,graphname="test"):
    name=f"{graphname}.txt"
    with open(name, "w") as f:
        sys.stdout = f

        output_to_file(G,total_data,ordering_trees)

        sys.stdout = sys.__stdout__
    return

In [39]:
def run_example(graphname,G,edge_labels):
    ordering_trees=list(G.spanning_trees())
    subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs T\cup e.-D

    #Get assignment,lbm for all wsc
    ass_datum=unpickle_assignment_datum(graphname) # the assignments are in the order ordering_trees.

    total_data=run_over_all_assignments(ass_datum,G,edge_labels,ordering_trees,subgraphs)
    pickle_data(graphname,total_data)
    
    store_data(G,total_data,ordering_trees,graphname)
    print("Done")
    return total_data

# Determine the cycles for the following: <a name="s2"></a>

#### Vine blowups

In [ ]:
# graphname="K33"
# G=Graph([("0", "3"), ("0", "4"), ("0", "5"), ("1", "3"), ("1", "4"), ("1", "5"), ("2", "3"), ("2", "4"), ("2", "5")], multiedges=True)
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="K33_breaks"
# G=Graph([("0", "3"), ("0", "4"), ("0", "5"), ("1", "3"), ("1", "4"), ("1", "5"), ("2", "3"), ("2", "4"), ("2", "5")], multiedges=True)
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="V_222"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","4"),("4","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","0"):"e4",("0","4"):"e5",("4","2"):"e6"}

# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="V_223"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","5"),("5","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","0"):"e5",("0","5"):"e6",("5","2"):"e7"}

# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="V_224"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6",("0","6"):"e7",("6","2"):"e8"}

# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="V_233"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","3")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6",("0","6"):"e7",("6","3"):"e8"}

# run_example(graphname,G,edge_labels)

#### Higher genus graphs

In [ ]:
# graphname="G4M02M13"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2"),("1","3")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","0"):"e4",("0","2"):"e5",("1","3"):"e6"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G5M02M03"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","0"):"e5",("0","2"):"e6",("0","3"):"e7"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G5M02M03M14"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","0"):"e5",("0","2"):"e6",("0","3"):"e7",("1","4"):"e8"}
# run_example(graphname,G,edge_labels)

#### Genus 2 graphs with middle edges

In [43]:
# graphname="G4M2"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","0"):"e4",("0","2"):"e0"}
# run_example(graphname,G,edge_labels)

[([[0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  array([[0, 0, 1, 1],
         [0, 0, 2, 0],
         [0, 1, 0, 1],
         [0, 1, 1, 0],
         [1, 0, 0, 1],
         [1, 0, 1, 0],
         [1, 1, 0, 0],
         [2, 0, 0, 0]], dtype=int32),
  [(Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0]),
   (Graph on 4 vertices, [0, 0, 0, 0])],
  [(Multi-graph on 4 vertices, (1, 4, 3, 2), Multi-graph on 4 vertices),
   (Multi-graph on 4 vertices, (0, 4, 3), Multi-graph on 4 vertices),
   (Multi-graph on 4 vertices, (1, 0, 2), Multi-graph on 4 vertices),
   (Multi-graph on 4 vertices, (1, 0, 2), Multi-graph on 4 vertices),
   (Multi-graph on 4 vertices, (0, 4, 3), Multi-graph on 4 v

In [ ]:
# graphname="G5M2"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","0"):"e5",("0","2"):"e0"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G6M2"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","2")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6",("0","2"):"e0"}
# run_example(graphname,G,edge_labels)

In [54]:
graphname="G6M3"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3")], multiedges=True)
edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6",("0","3"):"e0"}

ordering_trees=list(G.spanning_trees())
subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs T\cup e.-D

#Get assignment,lbm for all wsc
ass_datum=unpickle_assignment_datum(graphname) # the assignments are in the order ordering_trees.

total_data=run_over_all_assignments(ass_datum,G,edge_labels,ordering_trees,subgraphs)
output_to_file(G,total_data,ordering_trees)

# run_example(graphname,G,edge_labels)

In [111]:
graphname="G6M3_non"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3")], multiedges=True)
edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6",("0","3"):"e0"}



# run_example(graphname,G,edge_labels)

# # def run_example(graphname,G,edge_labels):
ordering_trees=list(G.spanning_trees())
subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs T\cup e.-D
#Get assignment,lbm for all wsc
ass_datum=unpickle_assignment_datum(graphname) # the assignments are in the order ordering_trees.

In [112]:
# ass_datum[7]
# ass_datum[0]
# ass_datum[1]
# ass_datum=[ass_datum[7]]

ass_datum=[ass_datum[3]]

In [113]:
total_data=run_over_all_assignments(ass_datum,G,edge_labels,ordering_trees,subgraphs)
output_to_file(G,total_data,ordering_trees)    
    
#     pickle_data(graphname,total_data)
    
#     store_data(G,total_data,ordering_trees,graphname)
#     print("Done")
#     return total_data

IndexError: list index out of range

#### Cyclic graphs: genus 1

In [ ]:
# graphname="G3"
# G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","0"):"e3"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G4"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","0"):"e4"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G5"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","0"):"e5"}
# run_example(graphname,G,edge_labels)

In [ ]:
# graphname="G6"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0")], multiedges=True)
# edge_labels = {("0","1"):"e1",("1","2"):"e2",("2","3"):"e3",("3","4"):"e4",("4","5"):"e5",("5","0"):"e6"}
# run_example(graphname,G,edge_labels)

#### Graphs we do not have

In [ ]:
# graphname="V_225"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","2")], multiedges=True)
run_example(graphname,G,edge_labels)
# graphname="V_234"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","3")], multiedges=True)
run_example(graphname,G,edge_labels)
# graphname="V_235"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","8"),("8","2")], multiedges=True)
run_example(graphname,G,edge_labels)
# graphname="V_333"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","7"),("7","3")], multiedges=True)
run_example(graphname,G,edge_labels)
# graphname="V_334"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","8"),("8","3")], multiedges=True)
run_example(graphname,G,edge_labels)

# graphname="G8M04"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","7"),("7","0"),("0","4")], multiedges=True)
# run_example(graphname,G,edge_labels)
# graphname="G8M02"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","7"),("7","0"),("0","2")], multiedges=True)
# run_example(graphname,G,edge_labels)

# Used Throughout

In [40]:
import numpy as np
import networkx as nx
from collections import Counter
import itertools 
import random
from itertools import combinations, permutations
import sys
import pickle

# from sage.graphs.connectivity import connected_components_subgraphs

In [41]:
import ast
import inspect

def find_functions_used(func):
    
    """Add print Objective."""
    
    functions_used = set()
    tree = ast.parse(inspect.getsource(func))
    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            functions_used.add(node.func.id)
    return functions_used

def find_functions_used_l(l):
    for func in l:
        functions_used = find_functions_used(func)
        print(f"{func.__name__}:{functions_used} \n")

# l = [f1, f2, f3]
# find_functions_used_l(l)


# Leftovers

In [ ]:
# def get_Im_data(case,subg): #OLD
#     """
#         The following function returns the assignment data we want on Im part for the subnecklace.
#         Using this we can get the cycle.

#         Returns: relabler too 
#     """
#     # Pick subg
#     vert_num_G=len(list(subg.vertices()))

#     # What is Im necklace
#     [edges_subnecklace]=subg.cycle_basis(output='edge') #return cycle of subgnecklace
#     Im=Graph(edges_subnecklace)
#     m=len(edges_subnecklace) #length of subnecklace

#     # Get spanning trees of subg
#     subg_trees=list(subg.spanning_trees())

#     #Data we want to analyse.

#     ## Pick (T,Ass) from ass_datum for trees in subg_trees.
#     data=[]
#     for pair in case:
#         if pair[0] in subg_trees:
#             data.append(pair)

#     ## Pick (t,ass_t) where we will have ass_t=0.
#     common_tree,common_tree_assingment=data[0] # tree with tails still

#     # From data we need to remove the tails from the trees.

#     ## Remove disconnected vertices from tails.
#     tails=get_tails(subg)
#     tails=remove_discon_verts(tails)

#     ## Remove tails from trees of subg in data.
#     removed_tails_data=remove_tails_tree_of_subg(data,tails)
#     data=removed_tails_data

#     # Label (t,ass_t) where we have ass_t=0 in Im.
#     Im_common_tree,Im_common_tree_assingment=data[0]

#     ## Relabel trees of data in Subg necklace format to In format.
#     relabler=get_relabler(subg,Im_common_tree,leave_order=None)
#     temp_data=get_temp_data(relabler,data)


#     # Consider the assignments and translate so common_tree_assingment=0.
#     Im_data=un_translate_data(temp_data,common_tree_assingment)

#     ## Remove components of assignments from vertices of tail of subg from assignments of trees in subg.

#     Im_data=get_Im_assignments(relabler,Im_data,Im_common_tree)

#     return Im_data,relabler

In [ ]:
# #Compare the following with check in analysis_cycles
# dd=examp[2][3]
# dd

# #Lets check we have translator trees for each subg.

# ll=[i[2] for i in dd]
# graphs_list.show_graphs(ll)
# #subgraph
# ll=[i[0] for i in dd]
# graphs_list.show_graphs(ll)

In [ ]:
## Example all details worked out

# # #example all worked out

# # #Take example case-D
# # example=assignment_data_all[8]

# # #Get #\tilde{G} the subgraphs T\cup e.-D
# # subgraphs= get_subgraphs_gm1_Edgesmissing(G) 

# # #pick example-D
# # subg=subgraphs[5]
# vert_num_G=len(list(subg.vertices()))
# show(subg)

# # get spanning trees of subg-D
# subg_trees=list(subg.spanning_trees())


# #pick (T,Ass) from ass_datum for trees in subg_trees.
# subg_datum=[]
# for pair in example:
#     if pair[0] in subg_trees:
#         subg_datum.append(pair)
        
# data=subg_datum

# # pick one to be common_tree
# common_tree,common_tree_assingment=data[0] # tree with tails still
# show(common_tree)

# #From data we need to remove the tails from the trees.
# removed_tails_data=remove_tails_tree_of_subg(data,tails)
# data=removed_tails_data
# Im_common_tree,Im_common_tree_assingment=data[0]
# show(Im_common_tree)


# relabler=get_relabler(subg,Im_common_tree,leave_order=None)

# temp_data=get_temp_data(relabler,data)

# print("Before relabling")
# show(Im_common_tree)

# print("Relabeler:",relabler)

# print("After relabling")
# show(temp_data[0][0])

# Now the trees of data are in the correct format of Im

# Im_data=un_translate_data(temp_data,common_tree_assingment)

# Im_data

# Im_data[3][0]

# # find_leaves(data[3][0])#[3, 4]

# Im_data=get_Im_assignments(relabler,Im_data,Im_common_tree)
# Im_data

# Im_data=get_Im_data(case,subg)
# n=int(5)
# check(Im_data,n)

# #Order of trees in Im_data

# print("#Order of trees in Im_data")
# for d in Im_data:
#     print(d[0].edges(),d[1])
# print("\n")

# #What is the length of the subnecklace?
# # n=len(edges_subnecklace)#length of subnecklace
# cycles=get_cycles(n)
# #example
# cycle=cycles[5]
# #(tree,ass) data for this cycle in In case.
# cycle_data=change_to_get_trees(cycle)

# #get list of trees for In case.
# print("list of trees for In case.")
# cycle_trees=[d[0] for d in cycle_data]

# for d in cycle_data:
#     print(d[0].edges(),d[1])
    
# # get list of assignments: it is this we want to check if it is equal 
# cycle_assignments=[d[1] for d in cycle_data]

# Im_data

# #Put Im_data into In case format.
# print("\n Im_data in In case format.")
# Im_data=get_ordered_list(Im_data,cycle_trees)

# for d in Im_data:
#     print(d[0].edges(),d[1])
    
# Im_data_assignments=[d[1] for d in Im_data]

# check(Im_data,n)